### Create a SQL DB from a series of processed CSV files via SQLAlchemy.

The processed CSV files were imported from the WHO, UN, OECD, World Bank, loaded, processed and exported via Pandas


In [44]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Create a list (array) with the names of the CSV files from the processed data location

In [45]:
datapath = os.path.join(os.getcwd(), "data2")

if os.path.isdir(datapath):
    datafiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]

For these files, split the extension, capitalize and append them to a dictionary which will contain as key the name minus extension and as value the Pandas DataFrame

In [46]:
datadict = dict()

for f in datafiles:
    #print(f)
    datadict[((f.rsplit(".", 1)[0]).capitalize())] = pd.read_csv(os.path.join(datapath, f), index_col = "Country")

In [47]:
for key, val in datadict.items():
    print(key)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [48]:
# find for each DB, the unique country/index entries
# which we convert to lists and then sets to find the
# intersection of the lists/sets

uniqndx = []

for key, val in datadict.items():
    uniqndx.append(list(datadict[key].index.unique()))

In [49]:
#inter = [i for i in uniqndx[0]]
countries = []

for i, val in enumerate(uniqndx):
    countries = list(set(countries).intersection(set(uniqndx[i]))) \
        if i != 0 else [i for i in uniqndx[0]]

In [50]:
for key, val in datadict.items():
    print(key)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [51]:
df = pd.DataFrame()

### Check for NaNs

we're going to have NaNs since multiple dataframes span different time ranges. Since we do an union of the dataframes we're guaranteed to have some fields/columns without data depending on the year.
If there are any *Code* fields left with NaN, fill to *NUL*. For everything else fill them with 0.


In [9]:
# NEEDED for geo maps

for key, val in datadict.items():
    print(key)
    datadict[key].drop(columns=["Code"], inplace = True)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [10]:
df = pd.merge(left=datadict["Anual_number_of_deaths_by_cause"], right=datadict["Average_total_years_of_schooling_for_adult_population"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_satisfaction_in_cantril_ladder_world_happiness_report_2021"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_expectancy"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_expectancy_at_birth"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Human_development_index"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Mortality_rate_under_5_per_1000_live_births"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Suicide_mortality_rate_per_100000_population"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Annual_co2_emissions"], how="outer", on=["Year", "Country"])

df.head()

,Year,Number_of_executions_(amnesty_international),Deaths_-_meningitis,Deaths_-_lower_respiratory_infections,Deaths_-_intestinal_infectious_diseases,Deaths_-_protein-energy_malnutrition,Terrorism,Deaths_-_cardiovascular_diseases,Deaths_-_alzheimer_disease_and_other_dementias,Deaths_-_chronic_kidney_disease,...,Life_satisfaction_in_Cantril_Ladder_(World_Happiness_Report_2021),Life_expectancy,Life_expectancy_at_birth,$1.90_per_day_-_share_of_population_below_poverty_line,"Life_expectancy_at_birth,_total_(years)","Total_population_(Gapminder,_HYDE_&_UN)",Human_development_index,Mortality_rate_under_5_per_1000_live_births,Suicidy_mortality_rate_per_100000_population,Annual_CO2_emissions
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,0,6469.977091,22836.912346,295.382207,1607.703797,12.0,46498.085024,1959.215373,3155.356340,...,NaN,50.331,50.331,0.0,50.331,12412311.0,NaN,17.77,NaN,2603238.0
Afghanistan,1991,0,6347.158764,22325.633931,303.866598,1558.132316,68.0,46967.361037,1987.812878,3124.082631,...,NaN,50.999,50.999,0.0,50.999,13299016.0,NaN,17.17,NaN,2427366.0
Afghanistan,1992,0,6659.741428,23205.280750,317.750121,1617.721576,49.0,48355.558515,2025.102326,3192.278428,...,NaN,51.641,51.641,0.0,51.641,14485543.0,NaN,16.59,NaN,1379462.0
Afghanistan,1993,0,8068.386263,28229.715397,333.978331,1931.808553,0.0,50072.563899,2064.827903,3343.974014,...,NaN,52.256,52.256,0.0,52.256,15816601.0,NaN,16.04,NaN,1333055.0
Afghanistan,1994,0,9432.845127,32652.297630,348.419085,2351.629206,22.0,51416.815442,2101.020630,3480.299710,...,NaN,52.842,52.842,0.0,52.842,17075728.0,NaN,15.52,NaN,1281602.0


In [11]:
df.columns

Index(['Year', 'Number_of_executions_(amnesty_international)',
       'Deaths_-_meningitis', 'Deaths_-_lower_respiratory_infections',
       'Deaths_-_intestinal_infectious_diseases',
       'Deaths_-_protein-energy_malnutrition', 'Terrorism',
       'Deaths_-_cardiovascular_diseases',
       'Deaths_-_alzheimer_disease_and_other_dementias',
       'Deaths_-_chronic_kidney_disease',
       'Deaths_-_chronic_respiratory_diseases',
       'Deaths_-_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_-_digestive_diseases', 'Deaths_-_hepatitis',
       'Deaths_-_neoplasms', 'Deaths_-_parkinson_disease',
       'Deaths_-_fire,_heat,_and_hot_substances', 'Deaths_-_malaria',
       'Deaths_-_drowning', 'Deaths_-_interpersonal_violence',
       'Deaths_-_hiv/aids', 'Deaths_-_drug_use_disorders',
       'Deaths_-_tuberculosis', 'Deaths_-_road_injuries',
       'Deaths_-_maternal_disorders', 'Deaths_-_neonatal_disorders',
       'Deaths_-_alcohol_use_disorders',
       'Deaths_-_exposure

In [12]:
df.rename(columns={"Number_of_executions_(amnesty_international)" : "Number_of_executions", "Life_satisfaction_in_Cantril_Ladder_(World_Happiness_Report_2021)" : "Life_satisfaction", "$1.90_per_day_-_share_of_population_below_poverty_line" : "Share_of_population_below_poverty_line_2USD_per_day", "Total_population_(Gapminder,_HYDE_&_UN)" : "Total_population"}, inplace=True)

In [13]:
df.drop(columns={"Number_of_executions"}, inplace=True)
df.columns

Index(['Year', 'Deaths_-_meningitis', 'Deaths_-_lower_respiratory_infections',
       'Deaths_-_intestinal_infectious_diseases',
       'Deaths_-_protein-energy_malnutrition', 'Terrorism',
       'Deaths_-_cardiovascular_diseases',
       'Deaths_-_alzheimer_disease_and_other_dementias',
       'Deaths_-_chronic_kidney_disease',
       'Deaths_-_chronic_respiratory_diseases',
       'Deaths_-_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_-_digestive_diseases', 'Deaths_-_hepatitis',
       'Deaths_-_neoplasms', 'Deaths_-_parkinson_disease',
       'Deaths_-_fire,_heat,_and_hot_substances', 'Deaths_-_malaria',
       'Deaths_-_drowning', 'Deaths_-_interpersonal_violence',
       'Deaths_-_hiv/aids', 'Deaths_-_drug_use_disorders',
       'Deaths_-_tuberculosis', 'Deaths_-_road_injuries',
       'Deaths_-_maternal_disorders', 'Deaths_-_neonatal_disorders',
       'Deaths_-_alcohol_use_disorders',
       'Deaths_-_exposure_to_forces_of_nature', 'Deaths_-_diarrheal_diseases',
 

In [14]:
df.columns = df.columns.str.replace("_-_", "_")

In [15]:
df.columns

Index(['Year', 'Deaths_meningitis', 'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein-energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_fire,_heat,_and_hot_substances',
       'Deaths_malaria', 'Deaths_drowning', 'Deaths_interpersonal_violence',
       'Deaths_hiv/aids', 'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutri

In [16]:
df.rename(columns={"Deaths_hiv/aids" : "Deaths_aids"}, inplace = True)

In [17]:
df.columns = df.columns.str.replace("-","_")

In [18]:
df.rename(columns={"Life_expectancy_at_birth,_total_(years)" : "Total_life_expectancy_at_birth_in_years"}, inplace = True)

In [19]:
df.rename(columns={"Deaths_protein-energy_malnutrition" : "Deaths_protein_energy_malnutrition"}, inplace = True)

In [20]:
df.rename(columns={"Deaths_fire,_heat,_and_hot_substances" : "Deaths_by_fire_and_heat"}, inplace=True)

In [21]:
df.columns

Index(['Year', 'Deaths_meningitis', 'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein_energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_by_fire_and_heat', 'Deaths_malaria',
       'Deaths_drowning', 'Deaths_interpersonal_violence', 'Deaths_aids',
       'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutritional_deficiencie

In [22]:
df.drop(columns="Unnamed: 0", inplace = True)

In [23]:
df.columns

Index(['Year', 'Deaths_meningitis', 'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein_energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_by_fire_and_heat', 'Deaths_malaria',
       'Deaths_drowning', 'Deaths_interpersonal_violence', 'Deaths_aids',
       'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutritional_deficiencie

In [24]:
df[df.isna().any(axis=1)]

,Year,Deaths_meningitis,Deaths_lower_respiratory_infections,Deaths_intestinal_infectious_diseases,Deaths_protein_energy_malnutrition,Terrorism,Deaths_cardiovascular_diseases,Deaths_alzheimer_disease_and_other_dementias,Deaths_chronic_kidney_disease,Deaths_chronic_respiratory_diseases,...,Life_satisfaction,Life_expectancy,Life_expectancy_at_birth,Share_of_population_below_poverty_line_2USD_per_day,Total_life_expectancy_at_birth_in_years,Total_population,Human_development_index,Mortality_rate_under_5_per_1000_live_births,Suicidy_mortality_rate_per_100000_population,Annual_CO2_emissions
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,6469.977091,22836.912346,295.382207,1607.703797,12.0,46498.085024,1959.215373,3155.356340,5954.958582,...,NaN,50.331,50.331,0.0,50.331,12412311.0,NaN,17.77,NaN,2.603238e+06
Afghanistan,1991,6347.158764,22325.633931,303.866598,1558.132316,68.0,46967.361037,1987.812878,3124.082631,6023.386323,...,NaN,50.999,50.999,0.0,50.999,13299016.0,NaN,17.17,NaN,2.427366e+06
Afghanistan,1992,6659.741428,23205.280750,317.750121,1617.721576,49.0,48355.558515,2025.102326,3192.278428,6217.245157,...,NaN,51.641,51.641,0.0,51.641,14485543.0,NaN,16.59,NaN,1.379462e+06
Afghanistan,1993,8068.386263,28229.715397,333.978331,1931.808553,0.0,50072.563899,2064.827903,3343.974014,6468.245470,...,NaN,52.256,52.256,0.0,52.256,15816601.0,NaN,16.04,NaN,1.333055e+06
Afghanistan,1994,9432.845127,32652.297630,348.419085,2351.629206,22.0,51416.815442,2101.020630,3480.299710,6678.495850,...,NaN,52.842,52.842,0.0,52.842,17075728.0,NaN,15.52,NaN,1.281602e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
World,1895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.485281e+09
World,1896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533711e+09
World,1897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.606311e+09


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37791 entries, Afghanistan to World
Data columns (total 45 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Year                                                   37791 non-null  int64  
 1   Deaths_meningitis                                      6686 non-null   float64
 2   Deaths_lower_respiratory_infections                    6686 non-null   float64
 3   Deaths_intestinal_infectious_diseases                  6686 non-null   float64
 4   Deaths_protein_energy_malnutrition                     6686 non-null   float64
 5   Terrorism                                              6686 non-null   float64
 6   Deaths_cardiovascular_diseases                         6686 non-null   float64
 7   Deaths_alzheimer_disease_and_other_dementias           6686 non-null   float64
 8   Deaths_chronic_kidney_disease            

In [26]:
df["Year"]
#df.dropna(inplace = True)

Country
Afghanistan    1990
Afghanistan    1991
Afghanistan    1992
Afghanistan    1993
Afghanistan    1994
               ... 
World          1895
World          1896
World          1897
World          1898
World          1899
Name: Year, Length: 37791, dtype: int64

In [27]:
df.dtypes

Year                                                       int64
Deaths_meningitis                                        float64
Deaths_lower_respiratory_infections                      float64
Deaths_intestinal_infectious_diseases                    float64
Deaths_protein_energy_malnutrition                       float64
Terrorism                                                float64
Deaths_cardiovascular_diseases                           float64
Deaths_alzheimer_disease_and_other_dementias             float64
Deaths_chronic_kidney_disease                            float64
Deaths_chronic_respiratory_diseases                      float64
Deaths_cirrhosis_and_other_chronic_liver_diseases        float64
Deaths_digestive_diseases                                float64
Deaths_hepatitis                                         float64
Deaths_neoplasms                                         float64
Deaths_parkinson_disease                                 float64
Deaths_by_fire_and_heat  

In [28]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///deadline_database_nonans_geo.db", echo=True)
sqlite_connection = engine.connect()

In [29]:
sqlite_table = "Deadline_database"

df.to_sql(sqlite_table, sqlite_connection, if_exists="replace")

2021-12-06 19:18:42,298 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Deadline_database")
2021-12-06 19:18:42,299 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 19:18:42,300 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Deadline_database")
2021-12-06 19:18:42,301 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 19:18:42,305 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Deadline_database" (
	"Country" TEXT, 
	"Year" BIGINT, 
	"Deaths_meningitis" FLOAT, 
	"Deaths_lower_respiratory_infections" FLOAT, 
	"Deaths_intestinal_infectious_diseases" FLOAT, 
	"Deaths_protein_energy_malnutrition" FLOAT, 
	"Terrorism" FLOAT, 
	"Deaths_cardiovascular_diseases" FLOAT, 
	"Deaths_alzheimer_disease_and_other_dementias" FLOAT, 
	"Deaths_chronic_kidney_disease" FLOAT, 
	"Deaths_chronic_respiratory_diseases" FLOAT, 
	"Deaths_cirrhosis_and_other_chronic_liver_diseases" FLOAT, 
	"Deaths_digestive_diseases" FLOAT, 
	"Deaths_hepatitis" FLOAT, 
	"Deaths_neoplasms" FLOAT, 
	"Deaths_park

In [30]:
# close connection to DB
sqlite_connection.close()

In [31]:
from pycountry import countries

In [40]:
countries.lookup("Canada").alpha_3

'CAN'

In [42]:
for c in df.index:
    df["Code"] = countries.lookup(c).alpha_3

LookupError: Could not find a record for 'andean latin america'

In [43]:
df.index.unique()

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa',
       'Andean Latin America', 'Andorra', 'Angola', 'Antigua and Barbuda',
       'Argentina', 'Armenia',
       ...
       'Europe (excl. EU-28)', 'French Equatorial Africa',
       'French West Africa', 'International transport', 'Kuwaiti Oil Fires',
       'Leeward Islands', 'North America (excl. USA)', 'Panama Canal Zone',
       'Ryukyu Islands', 'St. Kitts-Nevis-Anguilla'],
      dtype='object', name='Country', length=411)